# Task 1: Binary Representations

In this section, It creates some basic bit manipulation functions.
Trying to rotate bits to the left

References from https://github.com/ianmcloughlin/computational_theory/blob/main/materials/binary_representations.ipynb

# rotl(x,n=1), rotating left that rotates the bits in a 32-bit unsigned integer to the left n places.



In [13]:
# First try at rotate left... but I missed something

def rotl(x, n=1):
    return (x << n) | (x >> n)

# Testing with a simple number that only has 1 bit on
print(bin(rotl(0b00000000000000000000000000000001, 1)))


0b10


does not rotate correctly.
shifted right by n, but to wrap the bits properly I should shift right by 32 - n. Also, forgot to make sure we stay within 32 bits .

In [ ]:
# Fixed it by wrapping the bits properly and keeping the result to 32 bits.
# ref on https://github.com/ianmcloughlin/computational_theory/blob/main/materials/binary_representations.ipynb

def rotatel(x, n=1):

    return ((x << n) | (x >> (32 - n))) & 0xFFFFFFFF

# Testing again
print(bin(rotatel(0b00000000000000000000000000000001, 1)))  
print(bin(rotatel(0x80000000, 1)))  


0b10
0b1


# rotr (x,n=1) that rotates the bits in a 32-bit unsigned integer to the right n places.





this is basically the opposite of the last one.
rotating bits to the right, and wrapping it back to the left

ref : https://github.com/ianmcloughlin/computational_theory/blob/main/materials/binary_representations.ipynb

In [ ]:
# First attempt at rotating right – but something feels off

def rotater(x, n=1):
    return (x >> n) | (x << n)

print(bin(rotater(0b00000000000000000000000000000010, 1)))  # expecting 0b1


0b101


In [22]:
# Fixed it 
def rotater(x, n=1):
   
    return ((x >> n) | (x << (32 - n))) & 0xFFFFFFFF

print(bin(rotater(0b10, 1)))   
print(bin(rotater(0b1, 1)))    


0b1
0b10000000000000000000000000000000



the `return ((x >> n) | (x << (32 - n))) & 0xFFFFFFFF` function spins the bits to the right by n steps making sure everything stays within 32 bits  see https://github.com/ianmcloughlin/computational_theory/blob/main/materials/binary_representations.ipynb sections: bitwise shift , bitwise OR , bit masking , integer size

## ch ( x, y, z) that chooses the bits from y where x has bits set to 1 and bits in z where x has bits set to 0.

This function is like a little decision-maker.  
It looks at each bit of `x` — if it’s a 1, it picks the corresponding bit from `y`. If it’s a 0, it picks from `z`.

In [6]:
def ch(x, y, z):
    return (x & y) | (x & z)

print(bin(ch(0b1010, 0b1100, 0b0011))) # expecting 0b1010
print(bin(ch(0b1010, 0b1100, 0b0000))) # expecting 0b1000
print(bin(ch(0b1010, 0b0000, 0b0011))) # expecting 0b0000


0b1010
0b1000
0b10


In [7]:
# 
def ch(x, y, z):
   
    return (x & y) ^ (~x & z)

print(bin(ch(0b1010, 0b1100, 0b0011)))  # expecting 0b1010
print(bin(ch(0b1010, 0b1100, 0b0000)))  # expecting 0b1000
print(bin(ch(0b1010, 0b0000, 0b0011)))  # expecting 0b0000
print(bin(ch(0b1010, 0b1100, 0b1111)))  # expecting 0b1111

0b1001
0b1000
0b1
0b1101


The ch(x, y, z) function is based on bitwise operations covered in the Bitwise AND, OR, XOR, and NOT sections see ref: https://github.com/ianmcloughlin/computational_theory/blob/main/materials/binary_representations.ipynb combining them to make a per-bit logic - "if x then y else z"

## maj(x, y, z)  which takes a majority vote of the bits in x, y, and z.
The output should have a 1 in bit position i where at least two of x, y, and z have 1's in position i.
All other output bit positions should be 0.

This function is used in hashing algorithms to "vote" on each bit.
For each bit position `i`, it returns 1 if at least **two** of `x`, `y`, and `z` have a 1 in that position.

Example:  
- x = 1010  
- y = 1111  
- z = 0000  
→ maj = 1010 (only positions where at least two are 1)


In [ ]:
def maj(x, y, z): # majority function
    
    return x | y | z 

# Testing 
print(bin(maj(0b1010, 0b1100, 0b0011))) 


0b1111


In [ ]:
def maj(x, y, z):
    # Majority function: returns 1 if two or more of the bits are 1
    # and 0 otherwise.
    return (x & y) ^ (x & z) ^ (y & z) 

# Testing a few values:
x = 0b10101010 
y = 0b11110000
z = 0b00001111

print("x:", format(x, '08b')) 
print("y:", format(y, '08b'))
print("z:", format(z, '08b'))
print("maj:", format(maj(x, y, z), '08b'))


x: 10101010
y: 11110000
z: 00001111
maj: 10101010


### How the maj(x, y, z) logic works

The formula `(x & y) ^ (x & z) ^ (y & z)` works because it captures all the cases where **two or more** inputs have a 1.


- `x & y` → 1 only if both x and y have 1
- `x & z` → 1 if x and z do
- `y & z` → 1 if y and z do
Then XOR-ing all three gives us 1 in any position where **at least two** inputs have 1s.

results based on Bitwise (&) and XOR referenced in https://github.com/ianmcloughlin/computational_theory/blob/main/materials/binary_representations.ipynb 

- bitwise AND : talks how xx & y returns 1 only when both bits are 1.
- bitwise XOR : usefull when combining partial matches lin ( x & y ) ^ ( x & z) ^ (y & z).


# Task 2: Hash Functions

translation of a hash function from C into Python.



In [2]:
unsigned hash(char *s) {
    unsigned hashval;
    for (hashval = 0; *s != '\0'; s++)
        hashval = *s + 31 * hashval;
    return hashval % 101;
}


SyntaxError: invalid syntax (2215498579.py, line 1)

the goals is to 
- Convert the function to Python.

- Test it with a few strings.

- Explain why 31 and 101 are used.

- Include mistake version first, then correct version.

- Add natural markdown, comments, and commits.

In [13]:
def unsigned_hash(s):
    # Simple hash function that takes a string and returns a hash value.
    hashval = 0 # Initialize hash value to 0    
    for char in s: # Iterate over each character in the string
        hashval = hashval * 31 + ord(s) # Update hash value using the character's ASCII value
    return hashval % 101 # Return the hash value modulo 101 to keep it within a reasonable range
# Testing the simple_hash function
print(unsigned_hash("hello"))  


TypeError: ord() expected a character, but string of length 5 found

In [14]:
def unsigned_hash(s):
    
    hashval = 0 # Initialize hash value to 0
    for char in s: # Iterate over each character in the string 
        hashval = ord(char) + 31 * hashval # Update hash value using the character's ASCII value
    return hashval % 101 # Return the hash value modulo 101 to keep it within a reasonable range

# Testing the simple_hash function with various strings
print("Hash of 'hello':", unsigned_hash("hello"))
print("Hash of 'abc':", unsigned_hash("abc"))
print("Hash of 'hashing':", unsigned_hash("hashing"))
print("Hash of 'function':", unsigned_hash("function"))
print("Hash of 'test':", unsigned_hash("test"))
print("Hash of 'example':", unsigned_hash("example"))
print("Hash of 'data':", unsigned_hash("data"))


Hash of 'hello': 17
Hash of 'abc': 0
Hash of 'hashing': 25
Hash of 'function': 100
Hash of 'test': 86
Hash of 'example': 28
Hash of 'data': 55


## Conluding

## Why 31 and 101?

The unsigned_hash(s) function uses the logic described in see reference: https://github.com/ianmcloughlin/computational_theory/blob/main/materials/prime_numbers.ipynb 
- it multiplies by 31 (a small prime that can be computed efficiently using bit shifting) and uses modulo 101 (another prime) to keep the hash within a fixed range. The use of bitwise shift (<<) as a fast multiplication method see reference : https://github.com/ianmcloughlin/computational_theory/blob/main/materials/binary_representations.ipynb

# Task 3: SHA256 

Write a Python function that calculates the SHA256 padding for a given file.

- The function should take a file path as input.
- It should print, in hex, the padding that would be applied to it.
- The specification states that the following should be appended to a message:

- a 1 bit;
- enough 0 bits so the length in bits of padded message is the smallest possible multiple of 512;
- the length in bits of the original input as a big-endian 64-bit unsigned integer.

The example in the specification is a file containing the three bytes abc:

01100001 01100010 01100011

The output would be:

80 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00
00 00 00 00 00 00 00 00 18


In [107]:
def sha256_padding(file_path):
    with open(file_path, "rb") as f:
        data = f.read()

    bit_len = len(data) * 8
    padding = b'\x80'

    
    while (len(data) + len(padding) + 8) % 512 != 0:
        padding += b'\x00'

    bit_len_bytes = bit_len.to_bytes(8, 'big')
    padded = data + padding + bit_len_bytes

    print("Padded message (hex):", padded.hex())


In [139]:
def sha256_padding(file_path):

    with open(file_path, "rb") as f:
        data = f.read()

    bit_len = len(data) * 8  # total message length in bits
    padding = b'\x80'        # first padding byte: 10000000

    # Add 0x00 bytes until length ≡ 56 mod 64
    while (len(data) + len(padding) + 8) % 64 != 0:
        padding += b'\x00'

    # 64-bit big-endian length
    bit_len_bytes = bit_len.to_bytes(8, 'big')

    padded_msg = data + padding + bit_len_bytes
    full_padding = padding + bit_len_bytes

    print("Original data (hex):", data.hex())
    print("Padding (hex):", padding.hex())
    print("Length bytes (hex):", bit_len_bytes.hex())
    print("Full padded message (hex):", padded_msg.hex())
    print("Padded message length (bytes):", len(padded_msg))
    print("Padded message length (bits):", len(padded_msg) * 8)
    print("\nPadding Output:")
    print(" ".join(f"{byte:02x}" for byte in full_padding))



In [140]:
sha256_padding("abc.txt")


Original data (hex): 616263
Padding (hex): 8000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000
Length bytes (hex): 0000000000000018
Full padded message (hex): 61626380000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000018
Padded message length (bytes): 64
Padded message length (bits): 512

Padding Output:
80 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 18


The sha256_padding() function implements the SHA-256 message padding procedure mentioned in Motivation under Binary Representations (FIPS PUB 180-4 see reference: https://github.com/ianmcloughlin/computational_theory/blob/main/materials/binary_representations.ipynb) . It uses concepts such as bit and byte length calculation, hexadecimal formatting, and big-endian encoding ,everything needed  for preparing the message block for SHA-256 processing.

# Task 4: Prime Numbers

Calculate the first 100 prime numbers using two different algorithms.
Any algorithms that are well-established and works correctly are okay to use.
Explain how the algorithms work.

## Trial Division Method

This method checks each number starting from 2 and sees if it's divisible by any smaller number up to its square root.

It’s simple but slow, especially for large ranges. Still, it’s a good way to understand how primality checking works.

The concept of primes is mentioned in see reference: https://www.khanacademy.org/computing/computer-science/cryptography/comp-number-theory/a/trial-division


In [9]:
def is_prime(n):
    if n < 2:
        return False
    for i in range(2, int(n**0.5) + 1):  # optimization: sqrt(n)
        if n % i == 0:
            return False
    return True

def first_100_primes_trial():
    primes = []
    num = 2  # start from 2
    while len(primes) < 100:  # stop at 100
        if is_prime(num):
            primes.append(num)
        num += 1
    return primes

primes_trial = first_100_primes_trial()
print(primes_trial)
print("Count:", len(primes_trial))


[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541]
Count: 100


## Eratosthenes version - How It Works

The **Sieve of Eratosthenes** is a classic and much more efficient algorithm to generate a list of prime numbers.

Here's how it works:
1. Create a list of `True` values representing whether numbers from 0 to N are prime.
2. Set index 0 and 1 to `False` (they are not prime).
3. Start with the number 2 (the first prime).
4. For each number that is still marked as `True`, mark **all of its multiples** (starting from its square) as `False`.
   - Why start from the square? Because smaller multiples will have already been crossed out by smaller primes.
5. Continue until you've marked all non-primes.

This method avoids checking each number individually, and instead **eliminates composites in bulk**, which is much faster.

**Example** (first few steps):
- Start with 2 → cross out 4, 6, 8...
- Move to 3 → cross out 6, 9, 12...
- Move to next unmarked number (5) → cross out 10, 15, 20...

By the time you reach √N, all remaining `True` positions in the list are prime.

**Key advantages**:
- Much faster than trial division
- Perfect for generating many primes up to a known limit

This algorithm is described in, see reference: https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes ; https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes

In [15]:
def sieve_primes(limit):
    sieve = [True] * (limit + 1)
    sieve[0:2] = [False, False]
    primes = []
    for num in range(2, limit + 1):
        if sieve[num]:
            primes.append(num)
            for multiple in range(num * num, limit + 1, num):
                sieve[multiple] = False
        if len(primes) == 100:
            break
    return primes
primes_sieve = sieve_primes(600)
print(primes_sieve)
print("Total:", len(primes_sieve))


[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313, 317, 331, 337, 347, 349, 353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541]
Total: 100


# Task 5: Roots

Calculate the first 32 bits of the fractional part of the square roots of the first 100 prime numbers.

In [4]:
# Redefine the first_100_primes_trial function
def first_100_primes_trial():
	def is_prime(n):
		if n < 2:
			return False
		for i in range(2, int(n**0.5) + 1):
			if n % i == 0:
				return False
		return True

	primes = []
	num = 2
	while len(primes) < 100:
		if is_prime(num):
			primes.append(num)
		num += 1
	return primes

# Use the first_100_primes_trial function to get the first 100 primes
primes = first_100_primes_trial()

# Define the fractional_bits function
def fractional_bits(primes):
	fractional_parts = []
	for prime in primes:
		sqrt_fractional = (prime**0.5) % 1  # Get the fractional part of the square root
		first_32_bits = int(sqrt_fractional * (2**32))  # Extract the first 32 bits
		fractional_parts.append(first_32_bits)
	return fractional_parts

# Call the fractional_bits function with the primes
correct_bits = fractional_bits(primes)
print(correct_bits[:100])

# Output the first 5 fractional bits
print("First 5 fractional bits:", correct_bits[:5])
print("Total:", len(correct_bits))
print("Total primes:", len(primes))
# hexadecimal representation of the first 5 fractional bits
print("Hexadecimal representation of the first 5 fractional bits:")
for bits in correct_bits[:100]:
    print(hex(bits))

[1779033703, 3144134277, 1013904242, 2773480762, 1359893119, 2600822924, 528734635, 1541459225, 3418070365, 1654270250, 2438529370, 355462360, 1731405415, 2394180231, 3675008525, 1203062813, 2925498710, 3479995859, 796084093, 1830299338, 2336478295, 3814765974, 474308610, 1863934769, 3645816497, 214214161, 639484402, 1477814206, 1891102055, 2706455657, 1157183029, 1913507325, 3026663070, 3392277365, 887149614, 1237834173, 2276178418, 3294864124, 3963601907, 656899949, 1628171250, 1948300446, 3523054131, 3833017748, 153196534, 458427541, 2258461506, 4007997007, 285697673, 570139515, 1135321014, 1974073629, 2251313177, 3620569459, 134086911, 933187903, 1723224488, 1984608322, 2763025377, 3277294613, 3533056595, 503553856, 2239462382, 2728129246, 2971284200, 3455274624, 830669862, 1535707435, 2696964627, 2927199279, 3385697929, 4068602425, 675358098, 1345217755, 2009711121, 2449788317, 3105617507, 3972238493, 107307157, 960992129, 2016442007, 2226015100, 3266492245, 3473134094, 4090212631

The fractional_bits(primes) function replicates the constant generation process described in Section 4.2.2 of the FIPS PUB 180-4 specification. It extracts the first 32 bits of the fractional part of the square roots of the first 100 primes, a method used in SHA-256 to generate initial hash values and round constants. The primes are computed using trial division, which is also supported by widely recognized algorithms such as those described on Khan Academy: https://www.khanacademy.org/computing/computer-science/cryptography/comp-number-theory/a/trial-division and Wikipedia: https://en.wikipedia.org/wiki/Trial_division.

# Task 6 - Finding English Words with the Most Leading Zero Bits in SHA-256

**Objective:**  
Find the word(s) in the English language with the greatest number of 0 bits at the beginning of their SHA-256 hash digest.




In [25]:
import hashlib

with open("words.txt", "r") as f:
    words = [line.strip() for line in f if line.strip()]
    
print(f"Loaded {len(words)} words.")


Loaded 3000 words.
